# Naive Bayes Algorithm and Spam Detection
#### by Palermo Penano
This notebook implements and describes the Naive Bayes algorithm for spam detection. It works through the entire pipeline of retrieving the data, preprocessing, applying the Bag of Words model to create the features, training the model using the scikit-learn package, and evaluating the model using the appropriate metric for unbalanced classes.

I used the tutorial in this link as a guide: https://github.com/udacity/machine-learning/blob/master/projects/practice_projects/naive_bayes_tutorial/Naive_Bayes_tutorial.ipynb


In [40]:
import urllib
import pandas as pd

# for importing zipfiles from url
from requests import get
from io import BytesIO
from zipfile import ZipFile

# scikit-learn modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Get data and import into a Pandas dataframe

In [4]:
# download zipfile from website
data_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
request = get(data_path)
data_downloaded = BytesIO(request.content)
zip_file = ZipFile(data_downloaded)

# reading the first file in the zip file and decode raw bytes in the string 
spam_data = zip_file.read('SMSSpamCollection').decode('utf-8')

# write data to a file
with open('spam_data.txt', 'w') as f:
    f.write(spam_data)

In [5]:
# Import into a pandas dataframe
df = pd.read_csv('spam_data.txt', 
                 sep='\t',
                 header=None,
                 names=['label', 'sms_message']
                )
df.head()
print(df.shape)

(5572, 2)


## Convert string labels into binary variables

In [6]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Bag of words

Machine learning algorithms needs numeric features for training. To get numeric features from a collection of strings, use the Bag of Words model to convert the sentences into a series of columns containing the frequency of each word for each sentence in the data.

The steps to convert text into features using the Bag of Words model:
1. Convert all string into lower case form
2. Remove all punctuations
3. Tokenize sentences (split up sentences into individual words using a delimiter and save to list)
4. Count occurence of each word for each sentence using data generated in 3.

Example below demonstrates how to do this using the CountVectorizer class in sklearn.feature_extraction.text

In [7]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

count_vector = CountVectorizer(lowercase=True)

# learn vocabulary
count_vector.fit(documents)

# transform documents to a document-term matrix, then to an array
doc_array = count_vector.transform(documents).toarray()
print(doc_array)

# load into dataframe
feature_names = count_vector.get_feature_names()
frequency_matrix = pd.DataFrame(doc_array, 
                                columns=feature_names)
frequency_matrix.head()

[[1 0 0 1 0 1 0 0 0 0 0 1]
 [0 0 1 0 1 0 0 1 0 0 2 0]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 1 0 2 0 0 0 0 0 1 0 1]]


,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


## Split data into training and testing
Split training and label data into a training set (75%) and a test set (25%)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'],
                                                    df['label'],
                                                    random_state=1
                                                   )

print("Full data size:", df.shape[0])
print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])

Full data size: 5572
Training set size: 4179
Test set size: 1393


## Apply Bag of Words processing to the split data
We fit the vocabulary dictionary only for the training data. Here's a great explanation why we do this: https://sebastianraschka.com/faq/docs/scale-training-test.html. The basic idea is that we treat test data as new and unseen. Therefore, we cannot learn--by applying the fit() function--the vocabulary of this new data and must use the vocabulary learned from the training set.

In [18]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

Note that training_data and testing_data are sparse matrices containing the frequency distribution of each word from the learned vocabulary using the training data. To get the pandas dataframe, we must pass this into the DataFrame method in pandas

In [27]:
spam_feature_names = count_vector.get_feature_names()
spam_frequency_matrix = pd.DataFrame(training_data.toarray(), 
                                columns=feature_names)

print("Number of words in learned dictionary from training_data:", len(spam_feature_names))

Number of words in learned dictionary from training_data: 7456


## Bayes Theorem review

The example in the tutorial is on finding the odds a person will have diabetes given that they tested positive for it. Let the probability of having diabetes, denoted by $P(D)$, be 0.01. This is often referred to as the __base rate__ or the __prior__ probability. Assume that the test is %90 accurate. This means that the probability of testing positive given that you have diabetes is $P(Pos|D) = 0.9$. Alternatively, the probabily of testing negative given the person doesn't have diabetes is $P(Neg|~D) = 0.9$.

The probability we are interested in is

$$P(D|Pos) = \frac{P(Pos|D)P(D)}{P(Pos)}$$

$P(Pos)$ is the probability of testing positive regardless of whether you have diabetes or not. People without diabetes can still test positive because the test is imperfect. The formula for $P(Pos)$ is

$$P(Pos) = P(D)P(Pos|D) + P({\sim}D)P(Pos|{\sim}D)$$

Intuitively, $P(D)P(Pos|D)$ is the probability of testing positive _and_ having diabetes. $P(~D)P(Pos|~D)$ is the probability of still testing positive _and_ not having diabetes. Their sum, therefore, is just the probability of testing positive, whether or not you have diabetes.

Putting all of this together the probability of having diabetes given that you tested positive is

In [ ]:
prob_d = 0.01
prob_pos_d = 0.9
prob_neg_no_d = 0.9
prob_pos = prob_d * prob_pos_d + (1 - prob_d) * (1 - prob_neg_no_d)

prob_d_pos = (prob_pos_d * prob_d) / prob_pos

print(prob_d_pos)

## Naive Bayes

Despite the test being accurate 90% of the time, your probability of actually having diabetes is quite low. This is because the base rate incidence of having diabetes, independent of whether you tested positive or not, is only 1%. 

In this example, there is only feature with which we can use to predict whether you have diabetes. It is conceivable that many other features can predict the disease. To extend our example, suppose we also had data on whether your parents also had diabetes. The posterior probability is now as follows

$$P(D|Pos, FamHist) = \frac{P(Pos, FamHist|D)P(D)}{P(Pos, FamHist)}$$

where $FamHist$ is an indicator equal to 1 if either one of your mother or father had diabetes. $P(Pos, FamHist)$ is the joint probability of a given individual having some outcome of a a test result and their family history. Repeatedly applying conditional probabilities, we can rewrite the object in the numerator as follows

\begin{equation}
\begin{split}
P(Pos, FamHist|D)P(D) & = P(Pos, FamHist, D)  \\
 & = P(Pos|FamHist, D)P(FamHist, D)  \\
 & = P(Pos|FamHist, D)P(FamHist|D)P(D)
\end{split}
\end{equation}

This can get unwieldy if we had 3, or 4, or in most cases, 10 or more features. As a simplification, assume that features are _conditionally_ independent given that you have diabetes. In other words, $P(Pos|FamHist, D) = P(Pos|D)$. The formula then simplifies to

\begin{equation}
\begin{split}
P(Pos, FamHist|D)P(D) & = P(Pos|D)P(FamHist|D)P(D)
\end{split}
\end{equation}

This is much easier to work with because with N features the formula is just


\begin{equation}
\begin{split}
P(x_1,...,x_N|D)P(D) & = \prod_{i=1}^{N}P(x_i|D)P(D)
\end{split}
\end{equation}

which is an formula to work with than if we didn't make the conditional independence assumption. 

This assumption is what makes this formulation of the Bayes Theorem naive. It is naive because it is conceivable that ones family history of diabetes, for whatever reason, affects the outcome of the test results. Perhaps this is not the best example for this, but imagine we are trying to predict whether house price will be high or low (based on some exogenously established threshold) given the number of rooms and the size of the house in square feet. The size of your house certainly affects how many rooms your house will have, thus, it unreasonable to assume that number of rooms is conditionally independent of house size. Making this assumption is the price we pay to make our formula tractable.

Going back to the diabetes example, the final formula of the posterior probability given conditionally independent features is

$$P(D|Pos, FamHist) = \frac{P(Pos|D)P(FamHist|D)P(D)}{P(Pos, FamHist)}$$

The denominator is just $P(Pos, FamHist) = P(D)P(Pos,FamHist|D) + P({\sim}D)P(Pos,FamHist|{\sim}D)$

In summary, Naive Bayes is just an extension of Bayes Theorem where the features are conditionally indepedent.

## Naive Bayes implementation using scikit-learn

Let's go back to our original problem of spam classification and train a multinomial naive bayes model using the training data. In our application, the features are the frequency of each word learned by when we applied the Bag of Words model to create our training data. There are 7456 words, which means we have 7456 features.

In [28]:
# train naive bayes classifier using training data
# training data is a sparse matrix
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [29]:
# make predictions using test data
predictions = naive_bayes.predict(testing_data)

## Model evaluation

There are four metrics we can use to evaluate the effectiveness of our model. The metric we use depends on how balanced our classifications are. The metrics are

1. Accuracy: Number of correct predictions / total number of predictions
2. Precision: Number of correct predictions / total number of examples predicted to be spam
3. Recall: Number of correct predicitons / total number of examples that are actually spam
4. F1 Score: harmonic mean of Precision and Recall

With highly unbalanced classification, accuracy is a poor metric. If out of 100 examples 3 are spam, we can predict every example to be not spam, in which case, we achieve an accuracy rating of 97%. Recall, however, would be zero (and precision, infinite). Because of the trade off between precision and recall, a better metric would be to use the F1 score.

In [37]:
y_train.describe()

count    4179.000000
mean        0.134482
std         0.341210
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [38]:
y_train.value_counts()

0    3617
1     562
Name: label, dtype: int64

Looking at the distribution of our classification, only about 13.5% of our messages are labeled spam. This suggests that we should consider using the F1 Score as our metric. We calculate them all any way for comparison.

In [39]:
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562
